#Exportar modelo de TensorFlow-Keras para TensorFlow.js
Fuente: https://www.tensorflow.org/js/tutorials/conversion/import_keras

In [ ]:
#@title Instalar parquete TensorFlow.js
#@markdown (necesario para exportar el modelo entrenado)
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalle

In [ ]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [ ]:
#@title Exportar Modelo y convertirlo para TensorFlow.js

import os.path
import shutil
from tensorflow import keras
from tensorflow.keras.models import load_model

path_modelo = '/content/gdrive/My Drive/IA/demoModelDeployment/modelo'  #@param {type:"string"}

# convierte el modelo como .H5 para hacer la conversión
model = load_model(path_modelo)
print("\nModelo cargado de ", path_modelo,"\n")

# lo guarada como si fuera .H5 para hacer la conversión
path_modeloH5 = "/content/modelo.h5"
model.save(path_modeloH5)
print("\nModelo H5 guardado en ", path_modeloH5,"\n")

if os.path.exists(path_modeloH5):
  path_modeloJS = "/content/modeloJS"
  #tfjs.converters.save_keras_model(model, path_modelo)
  !tensorflowjs_converter --input_format keras {path_modeloH5} {path_modeloJS}
  print("\nModelo exportado para TF.js en ", path_modeloJS,"\n")

  if os.path.exists(path_modeloJS):
    # copia archivos generados al drive
    path_modeloJS_drive = path_modelo + "/modeloJS"
    shutil.copytree(path_modeloJS, path_modeloJS_drive, dirs_exist_ok=True)
    drive.mount('/content/gdrive', force_remount=True)
    if os.path.exists(path_modeloJS_drive):
      print("\nArchivos de modelos movidos dentro de ", path_modeloJS_drive,"\n")


Modelo cargado de  /content/gdrive/My Drive/IA/DemoModelDeployment/modelo 


Modelo H5 guardado en  /content/modelo.h5 



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2023-11-28 13:59:20.564247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 13:59:20.564318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 13:59:20.565509: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 13:59:21.828325: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Modelo exportado para TF.js en  /content/modeloJS 

Mounted at /content/gdrive

Archivos de modelos movidos dentro de  /content/gdrive/My Drive/IA/DemoModelDeployment/modelo/modeloJS 



In [ ]:
#@title Reemplazar en JSON className "Functional" por "Model" (sino falla al cargar con TF.js)

import json
import re


fileJS = path_modeloJS_drive + '/model.json'
if os.path.exists(fileJS):

  # Opening JSON file
  with open(fileJS, 'r') as f:
    # carga datos
    data = json.load(f)

  # reemplaza valores
  print("Valor anterior: ", data['modelTopology']['model_config']['class_name'] )
  data['modelTopology']['model_config']['class_name'] = "Model"
  print("Nuevo valor: ", data['modelTopology']['model_config']['class_name'] )

  with open(fileJS, 'w') as f:
    # graba datos
    json.dump(data, f)

  print("\nDatos reemplazados en JSON ", fileJS,"\n")
else:
  print("\nNo se encuentra JSON ", fileJS, "\n")

Valor anterior:  Functional
Nuevo valor:  Model

Datos reemplazados en JSON  /content/gdrive/My Drive/IA/DemoModelDeployment/modelo/modeloJS/model.json 



In [ ]:
#@title Obtener IDs de archivos del Modelo TF.js en el Drive

!pip install kora
print("\n")
from kora.xattr import get_id

for x in [["FileId_modelJS", "model.json"], ["FileId_modelWeights", "group1-shard1of1.bin"]]:
  var = x[0]
  fn = path_modeloJS_drive + "/" + x[1]
  print("\n//- ", fn, ":")
  print("const " + var + " = \"" + get_id(fn) + "\"")





//-  /content/gdrive/My Drive/IA/DemoModelDeployment/modelo/modeloJS/model.json :
const FileId_modelJS = "1-8i5eQuR5WNVcRV5hAqUZP_aCgcGYvaW"

//-  /content/gdrive/My Drive/IA/DemoModelDeployment/modelo/modeloJS/group1-shard1of1.bin :
const FileId_modelWeights = "1-6ArkIvbosDQOCbwJnntitqe-xTHdBsy"
